In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

# Define the original cage (as a list of 2D points)
cage = np.array([
    [100, 100],
    [300, 100],
    [300, 300],
    [100, 300]
], dtype=np.float32)

# Move the cage to create deformation
deformed_cage = np.array([
    [80, 120],
    [320, 80],
    [280, 320],
    [120, 280]
], dtype=np.float32)


In [ ]:
def compute_mvc(p, cage):
    n = len(cage)
    w = np.zeros(n)
    eps = 1e-10

    for i in range(n):
        vi = cage[i] - p
        vip1 = cage[(i + 1) % n] - p

        len_vi = np.linalg.norm(vi)
        len_vip1 = np.linalg.norm(vip1)

        if len_vi < eps or len_vip1 < eps:
            # p is on a cage vertex
            w[i] = 1
            return w

        theta = np.arccos(
            np.clip(np.dot(vi, vip1) / (len_vi * len_vip1), -1.0, 1.0)
        )
        tan_half_theta = np.tan(theta / 2.0)
        w[i] += (tan_half_theta / len_vi)
        w[(i + 1) % n] += (tan_half_theta / len_vip1)

    w_sum = np.sum(w)
    return w / w_sum

def warp_image(image, cage, deformed_cage):
    h, w = image.shape[:2]
    warped_image = np.zeros_like(image)
    
    for y in range(h):
        for x in range(w):
            p = np.array([x, y], dtype=np.float32)
            if cv2.pointPolygonTest(cage, (x, y), measureDist=False) >= 0:
                weights = compute_mvc(p, cage)
                new_p = np.sum(weights[:, None] * deformed_cage, axis=0)
                new_x, new_y = new_p.astype(int)
                if 0 <= new_x < w and 0 <= new_y < h:
                    warped_image[y, x] = image[new_y, new_x]
    return warped_image


In [ ]:
import cv2
import numpy as np

def generate_test_image(width=400, height=400):
    # Create a blank white RGB image
    image = np.ones((height, width, 3), dtype=np.uint8) * 255

    # Draw a filled blue rectangle
    cv2.rectangle(image, (100, 100), (300, 300), (255, 0, 0), -1)

    # Add some text
    cv2.putText(image, 'MVC', (140, 220), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 3)

    return image


In [ ]:
image = generate_test_image()
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

warped = warp_image(image, cage, deformed_cage)

plt.subplot(1, 2, 1)
plt.imshow(image)
plt.title("Original")

plt.subplot(1, 2, 2)
plt.imshow(warped)
plt.title("Deformed")
plt.show()